<a href="https://colab.research.google.com/github/Ramkuchana/LLMs/blob/master/Fine-tuning%20Representation%20Models%20for%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning Representation Models for Classification

In [1]:
# %%capture
!pip install datasets>=2.18.0 transformers>=4.38.2 sentence-transformers>=2.5.1 setfit>=1.0.3 accelerate>=0.27.2 seqeval>=1.2.2

## **Data**

In [40]:
from datasets import load_dataset, DatasetDict, concatenate_datasets

# Load the original IMDb dataset
imdb_dataset = load_dataset("imdb")

# Extract the train and test datasets (exclude unsupervised)
train_dataset = imdb_dataset['train']
test_dataset = imdb_dataset['test']

# Filter positive and negative reviews from the train dataset
positive_train = train_dataset.filter(lambda x: x['label'] == 1)
negative_train = train_dataset.filter(lambda x: x['label'] == 0)

# Sample 4000 positive and 4000 negative reviews for a balanced train dataset
positive_train_sample = positive_train.shuffle(seed=42).select(range(4000))
negative_train_sample = negative_train.shuffle(seed=42).select(range(4000))

# Combine positive and negative samples to create a balanced train dataset
balanced_train = concatenate_datasets([positive_train_sample, negative_train_sample])

# Filter positive and negative reviews from the test dataset
positive_test = test_dataset.filter(lambda x: x['label'] == 1)
negative_test = test_dataset.filter(lambda x: x['label'] == 0)

# Sample 1000 positive and 1000 negative reviews for a balanced test dataset
positive_test_sample = positive_test.shuffle(seed=42).select(range(1000))
negative_test_sample = negative_test.shuffle(seed=42).select(range(1000))

# Combine positive and negative samples to create a balanced test dataset
balanced_test = concatenate_datasets([positive_test_sample, negative_test_sample])

# Create a new DatasetDict with the balanced train and test datasets
balanced_imdb_dataset = DatasetDict({
    'train': balanced_train,
    'test': balanced_test
})

# Verify the sizes of the new datasets
print(balanced_imdb_dataset)

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [41]:
from datasets import load_dataset, DatasetDict, concatenate_datasets

# Load the IMDb dataset
imdb_dataset = load_dataset("imdb")

# Function to balance the dataset
def balance_dataset(dataset, label_col, num_samples):
    # Filter positive and negative examples
    positive_samples = dataset.filter(lambda example: example[label_col] == 1)
    negative_samples = dataset.filter(lambda example: example[label_col] == 0)

    # Subsample both to the desired number
    positive_samples = positive_samples.shuffle(seed=42).select(range(num_samples // 2))
    negative_samples = negative_samples.shuffle(seed=42).select(range(num_samples // 2))

    # Concatenate positive and negative examples to form a balanced dataset
    balanced_dataset = concatenate_datasets([positive_samples, negative_samples]).shuffle(seed=42)

    return balanced_dataset

# Create a balanced train and test dataset
balanced_train = balance_dataset(imdb_dataset["train"], "label", 8000)
balanced_test = balance_dataset(imdb_dataset["test"], "label", 2000)

# Create a new DatasetDict with the balanced datasets
new_imdb_dataset = DatasetDict({
    "train": balanced_train,
    "test": balanced_test
})

# Print the size of the new datasets
print(f"Balanced train dataset size: {len(new_imdb_dataset['train'])}")
print(f"Balanced test dataset size: {len(new_imdb_dataset['test'])}")


Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Balanced train dataset size: 8000
Balanced test dataset size: 2000


In [29]:
from datasets import load_dataset
imdb_data = load_dataset("imdb")
print(imdb_data["train"][0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [42]:
new_imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [43]:
train_data, test_data = new_imdb_dataset["train"], new_imdb_dataset["test"]

## **Supervised Classification**

### HuggingFace Trainer

In [44]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load Model and Tokenizer
model_id = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tokenize our data.

In [45]:
from transformers import DataCollatorWithPadding

# Pad to the longest sequence in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
   """Tokenize input data"""
   return tokenizer(examples["text"], truncation=True)

# Tokenize train/test data
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Define metrics.

In [46]:
import numpy as np
import evaluate


def compute_metrics(eval_pred):
    """Calculate F1 score"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    load_f1 = evaluate.load("f1")
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"f1": f1}

Train model.

In [47]:
from transformers import TrainingArguments, Trainer

# Training arguments for parameter tuning
training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=1,
   weight_decay=0.01,
   save_strategy="epoch",
   report_to="none"
)

# Trainer which executes the training process
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


In [48]:
trainer.train()

Step,Training Loss
500,0.307200


TrainOutput(global_step=500, training_loss=0.3071924743652344, metrics={'train_runtime': 769.6268, 'train_samples_per_second': 10.395, 'train_steps_per_second': 0.65, 'total_flos': 2084785113806400.0, 'train_loss': 0.3071924743652344, 'epoch': 1.0})

Evaluate results.

In [49]:
trainer.evaluate()

{'eval_loss': 0.20039550960063934,
 'eval_f1': 0.9266100848726909,
 'eval_runtime': 61.8452,
 'eval_samples_per_second': 32.339,
 'eval_steps_per_second': 2.021,
 'epoch': 1.0}

### Freeze Layers

In [54]:
# Load Model and Tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
# Print layer names
for name, param in model.named_parameters():
    print(name)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

In [56]:
for name, param in model.named_parameters():

     # Trainable classification head
     if name.startswith("classifier"):
        param.requires_grad = True

      # Freeze everything else
     else:
        param.requires_grad = False

In [57]:
# We can check whether the model was correctly updated
for name, param in model.named_parameters():
     print(f"Parameter: {name} ----- {param.requires_grad}")

Parameter: bert.embeddings.word_embeddings.weight ----- False
Parameter: bert.embeddings.position_embeddings.weight ----- False
Parameter: bert.embeddings.token_type_embeddings.weight ----- False
Parameter: bert.embeddings.LayerNorm.weight ----- False
Parameter: bert.embeddings.LayerNorm.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.query.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.query.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.key.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.key.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.value.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.value.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.weight ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.LayerNorm.weight ----- False
Parameter: bert.encoder.layer.0.attention.output

In [58]:
from transformers import TrainingArguments, Trainer

# Trainer which executes the training process
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)
trainer.train()

Step,Training Loss
500,0.684800


TrainOutput(global_step=500, training_loss=0.6847626953125, metrics={'train_runtime': 258.912, 'train_samples_per_second': 30.899, 'train_steps_per_second': 1.931, 'total_flos': 2084785113806400.0, 'train_loss': 0.6847626953125, 'epoch': 1.0})

In [59]:
trainer.evaluate()

{'eval_loss': 0.6705530881881714,
 'eval_f1': 0.5846645367412141,
 'eval_runtime': 61.2597,
 'eval_samples_per_second': 32.648,
 'eval_steps_per_second': 2.04,
 'epoch': 1.0}

### Freeze blocks 1-5

In [60]:
# We can check whether the model was correctly updated
for index, (name, param) in enumerate(model.named_parameters()):
     print(f"Parameter: {index}{name} ----- {param.requires_grad}")

Parameter: 0bert.embeddings.word_embeddings.weight ----- False
Parameter: 1bert.embeddings.position_embeddings.weight ----- False
Parameter: 2bert.embeddings.token_type_embeddings.weight ----- False
Parameter: 3bert.embeddings.LayerNorm.weight ----- False
Parameter: 4bert.embeddings.LayerNorm.bias ----- False
Parameter: 5bert.encoder.layer.0.attention.self.query.weight ----- False
Parameter: 6bert.encoder.layer.0.attention.self.query.bias ----- False
Parameter: 7bert.encoder.layer.0.attention.self.key.weight ----- False
Parameter: 8bert.encoder.layer.0.attention.self.key.bias ----- False
Parameter: 9bert.encoder.layer.0.attention.self.value.weight ----- False
Parameter: 10bert.encoder.layer.0.attention.self.value.bias ----- False
Parameter: 11bert.encoder.layer.0.attention.output.dense.weight ----- False
Parameter: 12bert.encoder.layer.0.attention.output.dense.bias ----- False
Parameter: 13bert.encoder.layer.0.attention.output.LayerNorm.weight ----- False
Parameter: 14bert.encoder.laye

In [61]:
# Load model
model_id = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Encoder block 10 starts at index 165 and
# we freeze everything before that block
for index, (name, param) in enumerate(model.named_parameters()):
    if index < 165:
        param.requires_grad = False


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [62]:
# We can check whether the model was correctly updated
for index, (name, param) in enumerate(model.named_parameters()):
     print(f"Parameter: {index}{name} ----- {param.requires_grad}")

Parameter: 0bert.embeddings.word_embeddings.weight ----- False
Parameter: 1bert.embeddings.position_embeddings.weight ----- False
Parameter: 2bert.embeddings.token_type_embeddings.weight ----- False
Parameter: 3bert.embeddings.LayerNorm.weight ----- False
Parameter: 4bert.embeddings.LayerNorm.bias ----- False
Parameter: 5bert.encoder.layer.0.attention.self.query.weight ----- False
Parameter: 6bert.encoder.layer.0.attention.self.query.bias ----- False
Parameter: 7bert.encoder.layer.0.attention.self.key.weight ----- False
Parameter: 8bert.encoder.layer.0.attention.self.key.bias ----- False
Parameter: 9bert.encoder.layer.0.attention.self.value.weight ----- False
Parameter: 10bert.encoder.layer.0.attention.self.value.bias ----- False
Parameter: 11bert.encoder.layer.0.attention.output.dense.weight ----- False
Parameter: 12bert.encoder.layer.0.attention.output.dense.bias ----- False
Parameter: 13bert.encoder.layer.0.attention.output.LayerNorm.weight ----- False
Parameter: 14bert.encoder.laye

In [63]:

# Trainer which executes the training process
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)
trainer.train()


Step,Training Loss
500,0.389800


TrainOutput(global_step=500, training_loss=0.38983621215820313, metrics={'train_runtime': 341.0391, 'train_samples_per_second': 23.458, 'train_steps_per_second': 1.466, 'total_flos': 2084785113806400.0, 'train_loss': 0.38983621215820313, 'epoch': 1.0})

In [64]:
trainer.evaluate()

{'eval_loss': 0.244190514087677,
 'eval_f1': 0.9073610415623435,
 'eval_runtime': 61.646,
 'eval_samples_per_second': 32.443,
 'eval_steps_per_second': 2.028,
 'epoch': 1.0}

## Few-shot Classification

In [65]:
# Load the IMDb dataset
imdb_dataset = load_dataset("imdb")

In [66]:
from setfit import sample_dataset

# We simulate a few-shot setting by sampling 15 examples per class
sampled_train_data = sample_dataset(imdb_dataset["train"], num_samples=15)

In [67]:
from setfit import SetFitModel

# Load a pre-trained SentenceTransformer model
model = SetFitModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [68]:
from setfit import TrainingArguments as SetFitTrainingArguments
from setfit import Trainer as SetFitTrainer

# Define training arguments
args = SetFitTrainingArguments(
    num_epochs=3, # The number of epochs to use for contrastive learning
    num_iterations=20  # The number of text pairs to generate
)
args.eval_strategy = args.evaluation_strategy

# Create trainer
trainer = SetFitTrainer(
    model=model,
    args=args,
    train_dataset=sampled_train_data,
    eval_dataset=test_data,
    metric="f1"
)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
# from setfit import SetFitTrainer

# # Create trainer
# trainer = SetFitTrainer(
#     model=model,
#     train_dataset=sampled_train_data,
#     eval_dataset=test_data,
#     metric="f1",
#     num_epochs=3, # The number of epochs to use for contrastive learning
# )

In [69]:
# Training loop
trainer.train()

***** Running training *****
  Num unique pairs = 1200
  Batch size = 16
  Num epochs = 3


Step,Training Loss,Validation Loss


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [70]:
# Evaluate the model on our test data
trainer.evaluate()

***** Running evaluation *****


{'f1': 0.90104662226451}

In [71]:
model.model_head

LogisticRegression()